**Глобальное и локальное выравнивания**


*Глобальное* - предполагаются эквивалентные последовательности, выравниваемые "от конца до конца".

*Локальное* - нет штрафов за gap'ы на концах. Обычно используется при выравнивании короткого прочтения на большой геном.

Алгоритм Нидльмана-Вунша  - про глобальное выравнивание.

**Формат .fa(fasta)**:  

*   используется для нуклеотидных и протеиновых последовательностей
*   содержит 2 строки: 

  1.   Hазвание(характеристика),начинается с '>'
  2.   Сама последовательность


Функция, позволяющая получить последовательсности из данные формата fasta: 

In [14]:
def raspakovkaFA(link):
  with open(link) as file:
    lines = file.readlines() #массив строк
  res = ''
  lines = lines[1:] #удаляем строку характеризующую последовательность
  for j in range(len(lines)):
    res += lines[j][:len(lines[j])-1] #записываем все строки в одну удаляя символы '\n'
  return res



Алгоритм Нидльмана-Вунша:
1.   Подсчет содердимого матрицы на основании оптимизации схожести строк (=> gap, mismatch < 0, match > 0)
2.   Восстановление схожего вида

В данном алгоритме, значение mismatch задается пользователем (по умолчанию = -2), по нему однозначно определяются значения match и gap

In [21]:
l1 = raspakovkaFA("line1.txt")
l2 = raspakovkaFA("line2.txt")


def vunsh(line1,line2):
  mismatch = input() #запрашиваем значение штрафа за несоотвествие нуклеотидов
  if len(mismatch) == 0: #если ничего не передали, то определим по умолчанию
    mismatch = -2
  else:
    mismatch = -abs(int(mismatch)) #передаваемое число может быть как < так и >0(пользователь запутался), мы работаем только с <0 по идее алгоритма
  gap = 0.5*mismatch
  len1 = len(line1) #вертикаль line2, горизонталь line1
  len2 = len(line2)

  def equal(x,y):
    if x == y:
      return -mismatch #match
    else:
      return mismatch

  def table(line1,line2):
    F = [[0 for _ in range(len1 + 1)] for _ in range(len2 + 1)]   #массив для заполнения, (+1) засчет вспомогательных ячеек
                                                                  #F[len2][len1] - редакционное расстояние строк
    for i in range(len2+1):   #заполняем крайние случаи, с учетом F[0][0] = 0
      F[i][0] = i*gap
    F[0] = [ j*gap for j in range(len1 + 1)] 
    for i in range(1, len2 + 1):
      for j in range(1, len1 + 1):
        F[i][j] = max(F[i-1][j-1] + equal(line1[j-1], line2[i-1]),
                      F[i][j-1] + gap, 
                      F[i-1][j] + gap)
    return F
      
  F = table(l1, l2)
  res = [[] for _ in range(2)] #будем восстанавливать вид выравнивания, проходя по таблице с конца
  x,y = len2,len1 #индексы текущего элемента 
  curr = F[x][y] #current
  diag = F[x-1][y-1]
  verh = F[x-1][y]
  levo = F[x][y-1]
  while x != 0 or y != 0:
    if curr == diag + equal(line1[y-1], line2[x-1]):
      res[0].append(line1[y-1])
      res[1].append(line2[x-1])
      x -= 1
      y -= 1
    elif curr == levo + gap: #пришли в текущую ячейку слева
      res[0].append(line1[y-1])
      res[1].append('_')
      y -= 1
    elif curr == verh + gap: #сверху
      res[0].append('_')
      res[1].append(line2[x-1])      
      x-=1
    curr = F[x][y] #меняем положение в ячейку из которой пришли => меняем относительные правые и левые элементы
    diag = F[x-1][y-1]
    verh = F[x-1][y]
    levo = F[x][y-1]
  res[0].reverse()
  res[1].reverse()
  print(*res[0])
  print(*res[1])

vunsh(l1,l2)


A T G _ _ C _ _ _ C G T C T G A T G T C G G A A T A C G G
G T G A A C A A A C G G C T G _ T _ T T T G C A _ G C C G
